In [1]:
#Dependancies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import matplotlib.animation

from scipy.integrate import solve_ivp
from scipy.interpolate import CubicSpline
from scipy.optimize import fsolve, least_squares, minimize
from scipy.stats import qmc 
from scipy.linalg import norm
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from scipy.signal import argrelmin, argrelmax
from numba import jit
from tabulate import tabulate
from itertools import product
from operator import itemgetter

In [2]:
# @jit
# def model(t, X, V_max, γ_Z, φ, g, ν_x, λ_P, λ_Z, λ_Z_hat, λ_E, δ, μ_V, μ_V_prime, 
#           μ_u, μ_r, μ_s, μ_P, μ_delta, μ_g, μ_Z, K_N, K_I, K_h, K_P, I_0, ω):
    
#     """The NPZV system, formulated with 7 state variables defined below. 
#     Defined to be used with the numerical integrating function, solve_ivp."""
    
#     N_n = X[0] #New nutrients (Nitrate)
#     N_r = X[1] #Recycled Nutrients (Ammonium)
#     P_U = X[2] #Susceptible Hosts
#     P_I = X[3] #Infected Hosts
#     Z   = X[4] #Zooplankton
#     V_I = X[5] #Internal viruses from host nucleotide recycling 
#                #and de novo nucleotide synthesis
#     V_E = X[6] #Free (extracellular) viruses 
    
#     alpha = (1 / 1.42e8) * 2.1e-10 * 1e6 # host conversion, mmol ml / NT m^3
#     beta  = (1 / 4e5) * 1.27e-15 * 1e6   # viral conversion, mmol ml / NT m^3
    
#     if V_I == 0:  ν_i = 0 #avoid divison by 0 when evaluating ν_i
#     else: ν_i = V_I / ((beta / alpha) * P_I + V_I)
     
#     #Useful abreviations
#     P = P_U + P_I
#     N = N_n + N_r
#     nutr_lim = N / (K_N + N)
     
#     if N == 0: nutr_lim, N = (0, 1) #avoid divison by 0 when evaluating nutrient ratios

#     light_lim = np.log((K_I + I_0 / K_I)) / K_h  #light limitation term
    
#     psi = V_max * nutr_lim * light_lim
    
#     #Defining the scaling factors that continuously reduce the affect of growth
#     #as state variables reach ecologically unrealistic densities (e.g. 1e-20)
    
#     thresh = 1e-8  #threshold for when growth damping begins
#     scales = []
    
#     for state in [P_U, P_I, Z, V_I, V_E]:
        
#         scale = 1
#         if state < thresh: scale = state / thresh 
#         scales.append(scale)
            
#     P_U_scale, P_I_scale, Z_scale, V_I_scale, V_E_scale = scales

#     #Uninfected Phytoplankton
#     P_U_growth  = P_U_scale * (P_U + P_I_scale * μ_u * P_I) * psi
    
#     P_U_grazing = P_U * g * Z_scale * Z / K_P
#     P_U_mort    = P_U * λ_P
#     P_U_adsorp  = P_U * P_I_scale * (1 - ν_x) * φ * μ_s * V_E
#     P_U_entrain = P_U * ω 
    
#     P_U_dt = P_U_growth - P_U_grazing - P_U_mort - P_U_adsorp - P_U_entrain
    
#     #Infected Phytoplankton
#     P_I_growth   = P_I_scale * (1 - P_U_scale * μ_u) * P_I * psi
#     P_I_adsorp   = P_I_scale * (1 - ν_x) * φ * μ_s * P_U * V_E
    
#     P_I_grazing  = P_I * Z_scale * g * Z / K_P
#     P_I_mort     = P_I * λ_P
#     P_I_nt_recyc = P_I * ν_i * μ_V
#     P_I_lysis    = P_I * ν_i * δ
#     P_I_entrain  = P_I * ω
    
#     P_I_dt = P_I_growth + P_I_adsorp - P_I_grazing \
#             - P_I_mort - P_I_nt_recyc - P_I_lysis - P_I_entrain
    
#     #Zooplankton
#     Z_assim   = Z * Z_scale * γ_Z * g * (P + V_I) / K_P
#     Z_mort    = Z * (λ_Z + λ_Z_hat * Z)
#     Z_entrain = Z * ω

#     Z_dt = Z_assim - Z_mort - Z_entrain
    
#     #Intracellular viruses
#     V_I_growth  = V_I_scale * (beta / alpha) * ν_i * (μ_V + (nutr_lim * μ_V_prime)) * P_I
#     V_I_adsorp  = V_I_scale * (beta / alpha) * ν_x * φ * μ_s * P_U * V_E
    
#     V_I_grazing = V_I * Z_scale * g * Z / K_P
#     V_I_lysis   = V_I * V_E_scale * ν_i * δ
#     V_I_h_mort  = V_I * λ_P
#     V_I_entrain = V_I * ω
    
#     V_I_dt = V_I_growth + V_I_adsorp - V_I_lysis \
#             - V_I_grazing - V_I_h_mort - V_I_entrain
    
#     #Extracellular viruses
#     V_E_h_mort  = V_E_scale * V_I * μ_r * λ_P

#     V_E_adsorp  = V_I_scale * (beta / alpha) * ν_x * φ * μ_s * P_U * V_E
#     V_E_mort    = λ_E * V_E
#     V_E_entrain = V_E * ω
    
#     V_E_dt = V_I_lysis + V_E_h_mort - V_E_adsorp - V_E_mort - V_E_entrain
    
    
#     #Nitrate
#     N_n_deplet    = (N_n / N) * (P_U_growth + P_I_growth)
#     N_n_intracell = V_I_scale * (N_n / N) * (beta / alpha) * ν_i * P_I * (nutr_lim * μ_V_prime)
#     N_n_entrain   = ω * N_n 
    
#     N_n_dt = - N_n_deplet - N_n_intracell - N_n_entrain
            
        
#     #Recycled nutrients
#     N_r_deplet    = (N_r / N) * (P_U_growth + P_I_growth)
    
#     nt_recyc      = ν_i * P_I * (1 - V_I_scale * beta / alpha) * μ_V
#     denovo_syn    = ν_i * P_I * V_I_scale * (N_r / N) * (beta / alpha) * (nutr_lim * μ_V_prime)
#     N_r_intracell = nt_recyc - denovo_syn
    
#     N_r_mort    = λ_P * (P + (1 - V_E_scale * μ_r) * V_I)
#     N_r_sloppy  = Z_scale * (1 - γ_Z) * g * Z * (P + V_I) / K_P
#     N_r_entrain = N_r * ω
    
#     N_r_remin  = (μ_P * N_r_mort) + V_E_mort + (μ_delta * P_I_lysis) \
#                 + (μ_g * N_r_sloppy) + (μ_Z * Z_mort) - N_r_entrain

#     N_r_export = (1 - μ_P) * N_r_mort + (1 - μ_delta) * P_I_lysis \
#                 + (1 - μ_g) * N_r_sloppy + (1 - μ_Z) * Z_mort
    
#     N_r_dt = - N_r_deplet + N_r_intracell + N_r_remin + N_r_export
            
    
#     return N_n_dt, N_r_dt, P_U_dt, P_I_dt, Z_dt, V_I_dt, V_E_dt


### Adjusted system model - accounting for new prevention processes and corrected damping terms

In [3]:
@jit
def model(t, X, V_max, γ_Z, φ, g, ν_x, λ_P, λ_Z, λ_Z_hat, λ_E, δ, μ_V, μ_V_prime, 
          μ_u, μ_r, μ_s, μ_P, μ_delta, μ_g, μ_Z, K_N, K_I, K_h, K_P, I_0, ω):
    
    """The NPZV system, formulated with 7 state variables defined below. 
    Defined to be used with the numerical integrating function, solve_ivp."""
    
    N_n = X[0] #New nutrients (Nitrate)
    N_r = X[1] #Recycled Nutrients (Ammonium)
    P_U = X[2] #Susceptible Hosts
    P_I = X[3] #Infected Hosts
    Z   = X[4] #Zooplankton
    V_I = X[5] #Internal viruses from host nucleotide recycling 
               #and de novo nucleotide synthesis
    V_E = X[6] #Free (extracellular) viruses 
    
    alpha = (1 / 1.42e8) * 2.1e-10 * 1e6 # host conversion, mmol ml / NT m^3
    beta  = (1 / 4e5) * 1.27e-15 * 1e6   # viral conversion, mmol ml / NT m^3
    
    if V_I == 0:  ν_i = 0 #avoid divison by 0 when evaluating ν_i
    else: ν_i = V_I / ((beta / alpha) * P_I + V_I)
     
    #Useful abreviations
    P = P_U + P_I
    N = N_n + N_r
    nutr_lim = N / (K_N + N)
     
    if N == 0: nutr_lim, N = (0, 1) #avoid divison by 0 when evaluating nutrient ratios

    light_lim = np.log((K_I + I_0 / K_I)) / K_h  #light limitation term
    
    psi = V_max * nutr_lim * light_lim
    
    #Defining the scaling factors that continuously reduce the affect of growth
    #as state variables reach ecologically unrealistic densities (e.g. 1e-20)
    
    thresh = [1e-14, 1e-14, 1e-14, 1e-14] #threshold for when growth damping begins
    scales = []
    
    # for i, state in enumerate([P_U, P_I, Z, V_I]):
    for i, state in enumerate([P_U, P_I, Z, P_I]): # <- setting V_I thresh to turn on 
                                                   # only when P_I thresh is on
        scale = 1
        if state < thresh[i]: scale = state / thresh[i] 
        scales.append(scale)
            
    P_U_scale, P_I_scale, Z_scale, V_I_scale = scales

    #only damping on P_U
    P_I_scale, Z_scale, V_I_scale = [1, 1, 1]
    
    #Correction term for P_I with low V_I
#     if P_I != 0:
#         V_P_thresh = 6e-7 # Threshold equivalent to 1/10th of a virus per host
#         recovered_correction = max(0, P_I - (V_I / V_P_thresh)) #/time_scale = 1 day

#     else:
#         recovered_correction = 0
        
    recovered_correction = 0
    
    #V_E minimum for adsportion damping
    V_E_min = 1e-10
    
    #Uninfected Phytoplankton
    P_U_growth  = P_U_scale * (P_U + P_I_scale * μ_u * P_I) * psi
    
    P_U_grazing = P_U * g * Z_scale * Z / K_P
    P_U_mort    = P_U * λ_P
    P_U_adsorp  = P_U * (1 - ν_x) * φ * μ_s * max(0, V_E - V_E_min)
    P_U_entrain = P_U * ω 
    
    P_U_dt = P_U_growth - P_U_grazing - P_U_mort - P_U_adsorp \
             - P_U_entrain + recovered_correction
    
    #Infected Phytoplankton
    P_I_growth   = P_I_scale * (1 - P_U_scale * μ_u) * P_I * psi
    P_I_adsorp   = (1 - ν_x) * φ * μ_s * P_U * max(0, V_E - V_E_min)
    
    P_I_grazing  = P_I * Z_scale * g * Z / K_P
    P_I_mort     = P_I * λ_P
    P_I_nt_recyc = P_I * V_I_scale * ν_i * μ_V
    P_I_lysis    = P_I * ν_i * δ
    P_I_entrain  = P_I * ω
    
    P_I_dt = P_I_growth + P_I_adsorp - P_I_grazing \
            - P_I_mort - P_I_nt_recyc - P_I_lysis - P_I_entrain - recovered_correction
    
    #Zooplankton
    Z_assim   = Z * Z_scale * γ_Z * g * (P + V_I) / K_P
    Z_mort    = Z * (λ_Z + λ_Z_hat * Z)
    Z_entrain = Z * ω

    Z_dt = Z_assim - Z_mort - Z_entrain
    
    #Intracellular viruses
    V_I_nt_recyc   = V_I_scale * (beta / alpha) * ν_i * μ_V * P_I
    V_I_denovo_syn = V_I_scale * (beta / alpha) * ν_i * nutr_lim * μ_V_prime * P_I
    
    V_I_growth  = V_I_nt_recyc + V_I_denovo_syn
    V_I_adsorp  = (beta / alpha) * ν_x * φ * μ_s * P_U * max(0, V_E - V_E_min)
    
    V_I_grazing = V_I * Z_scale * g * Z / K_P
    V_I_lysis   = V_I * ν_i * δ
    V_I_h_mort  = V_I * λ_P
    V_I_entrain = V_I * ω
    
    V_I_dt = V_I_growth + V_I_adsorp - V_I_lysis \
            - V_I_grazing - V_I_h_mort - V_I_entrain
    
    #Extracellular viruses
    V_E_h_mort  = V_I * μ_r * λ_P

    V_E_adsorp  = (beta / alpha) * ν_x * φ * μ_s * P_U * max(0, V_E - V_E_min)
    V_E_mort    = λ_E * V_E
    V_E_entrain = V_E * ω
    
    V_E_dt = V_I_lysis + V_E_h_mort - V_E_adsorp - V_E_mort - V_E_entrain
    
    #Nitrate
    N_n_deplet    = (N_n / N) * (P_U_growth + P_I_growth) #scaling terms in previous def
    N_n_intracell = (N_n / N) * V_I_denovo_syn
    N_n_entrain   = ω * N_n 
    
    N_n_dt = - N_n_deplet - N_n_intracell - N_n_entrain
            
    #Recycled nutrients
    N_r_deplet    = (N_r / N) * (P_U_growth + P_I_growth) 
    
    N_r_nt_recyc   = V_I_scale * ν_i * P_I * (1 - beta / alpha) * μ_V
    N_r_denovo_syn = (N_r / N) * V_I_denovo_syn #has V_I_scale within V_I_denovo_syn
    
    N_r_intracell  = N_r_nt_recyc - N_r_denovo_syn
    
    N_r_mort    = λ_P * (P + (1 - μ_r) * V_I)
    N_r_sloppy  = Z_scale * (1 - γ_Z) * g * Z * (P + V_I) / K_P
    N_r_entrain = N_r * ω
    
    N_r_remin   = (μ_P * N_r_mort) + V_E_mort + (μ_delta * P_I_lysis) \
                + (μ_g * N_r_sloppy) + (μ_Z * Z_mort) - N_r_entrain

    N_r_export  = (1 - μ_P) * N_r_mort + (1 - μ_delta) * P_I_lysis \
                + (1 - μ_g) * N_r_sloppy + (1 - μ_Z) * Z_mort
    
    N_r_dt = - N_r_deplet + N_r_intracell + N_r_remin + N_r_export
            
    
    return N_n_dt, N_r_dt, P_U_dt, P_I_dt, Z_dt, V_I_dt, V_E_dt


### Perturb given initial condition arrays

In [4]:
def perturb_range(ranges, range_names, perturb=True, pprint=False):
    
    """Perturb each member of a list.
    Doesn't perturn last value..."""
    
    perturbed_ranges = []
    
    if perturb:
        
        for state_range in ranges:
            temp_list = state_range.copy()

            for i in range(len(temp_list)-1):
                range_gap = (temp_list[i + 1] - temp_list[i])
                pert = random.uniform(0, range_gap)

                temp_list[i] += pert

            #To increase reproducability, round ranges to 4 decimal places
            temp_list = np.round(temp_list, 4)
            perturbed_ranges.append(temp_list)
            
    else: 
        
        # perturbed_ranges = np.round(ranges) --> throws error since V_E_ratio
        # is different size than other state ranges
        for state_range in ranges:
            perturbed_ranges.append(np.round(state_range, 4))

    if pprint:
        fig, axs = plt.subplots(1, 4, figsize = (12, 3))
        
        for r, state_range in enumerate(ranges):
            
            axs[r].plot(state_range, 'o', color = 'blue', label = 'Original', alpha=.5)
            axs[r].plot(perturbed_ranges[r], 'o', color = 'red', label="Perturbed")

            axs[r].set_title(range_names[r] + " Range")
            axs[r].set_xlabel("Counter")
            axs[r].set_ylabel("Parameter Value")
            axs[r].set_yscale("log")
            axs[r].legend()
            axs[r].grid(alpha=.3)

        plt.tight_layout()
        plt.show()
        
    return perturbed_ranges

### Fancy print parameterization

In [5]:
def print_paramterization(param):
    
    """Print model parameters in a fancy table."""
    
    param_labels_fp = ['V_max', 'γ_Z', 'φ = 3.5e-10 / beta', 'g', 'ν_x = V_ind / (V_ind + S_ind)', 
                   'λ_P', 'λ_Z', 'λ_Z_hat', 'λ_E', 'δ', 'μ_V', 'μ_V_prime', 'μ_u', 'μ_r', 'μ_s', 
                   'μ_P', 'μ_delta', 'μ_g', 'μ_Z', 'K_N', 'K_I', 'K_h', 'K_P', 'I_0 = e - 1', 'ω']

    table = np.empty((5,5), dtype=object)
    i = 0

    for col in range(5):
        for row in range(5):

            table[row][col] = f"{param_labels_fp[i]} = {round(param[i], 4)}"
            i += 1

    print(tabulate(table, tablefmt='fancy_grid'))
    
    return

### Scheme for determining end state behavior

In [6]:
def ss_or_osc(sol):
    """
    sol: is solution object resulting from the solve_ivp integration
    state_sol: an individual soltuion for a particular state variable 
    associated with the larger solution set, sol.
    
    Convergence Criteria:

    - For state solution state_sol within the solution, set reference value to the 
      last time iterate of the state, state_sol[-1].
      
    - Work backwards, comparing each time iterate to the reference value: want to find 
      the first instance of when the state_sol has strayed too far from the reference,
      as determined by a specified threshold.

    - That first instance represents the time of convergence.
    - Example: l = 5 (solution with 5 time steps)
               ref = sol[-1] = sol[4]
               convergence at i = 3 yields sol[-4] = sol[2] = sol(5 - 3)

    """
    ss = []        
    conv_index = [] 
    
    for k in range(7): #looping through state variables
        
        state_sol = sol.y[k]
        #print(k)
        
        ### Check if oscillations occur:
        chunk = int(len(sol.t) * .85) #Last 15% of time span
        y_end = state_sol[chunk: ]    #look at tail end of specific state

        l_min = argrelmin(y_end)[0]  #Extracting local minimum 
        l_max = argrelmax(y_end)[0]  #and maximum

        if (len(l_min) != 0) or (len(l_max) != 0): #changed to "or" instead of "and"
            return 'osc' # Return identifier, exit loop
        

        ### If there are no oscillations, determine convergence time
        ref = state_sol[-1] #Reference value for which we compare the previous values to
        #print(ref)
        ss.append(ref)
        
        tol = max(ref * .01, 1e-10) # Define threshold at 1% of ref, while addressing exponential decay terms
    
        ### If reference is not zero, work backwards to find time of convergence
        index = 0
        l = len(state_sol) #Amount of time steps in a solution
            
        for i in range(2, l): # Work backwards comparing each time step to the ref
            if not math.isclose(state_sol[-i], ref, abs_tol=tol): 
                index = l - i + 1 #(plus 1 for previous time step, ie. last step that passed the threshold)
                break
        
        conv_index.append(index)
        
    return ss, conv_index

### Concentration to individual conversion

In [7]:
def est_viral_abund(r, P, alpha, beta, S_ind, V_ind):
    
    """
    Estimating viral abundance based off ratio of host to viral entitiy.
    
    r     (int)  : ratio of phytoplankton to viral enities, h_ind:v_ind
    P     (float): phytoplankton abundance
    alpha (float): conversion term for host related variables
    beta  (float): conversion term for virus related variables
    S_ind (float): Nucleotides per individual host
    V_ind (float): Nucleotides per individual virus
    """
    
    S_i = P / alpha / S_ind  #individual hosts per ml
 
    V_est = (1/r) * S_i * V_ind * beta #mmol N / m^3

    return V_est

In [8]:
#Threshold calculation example

alpha = (1 / 1.42e8) * 2.1e-10 * 1e6 # host conversion, mmol ml / NT m^3
beta  = (1 / 4e5) * 1.27e-15 * 1e6   # viral conversion, mmol ml / NT m^3

V_ind     = 4e5        # NT per individual virus, Table S1
S_ind     = 1.42e8     # NT per individual host, Table S1

est_viral_abund(1/10, 1e-14, alpha, beta, S_ind, V_ind)

6.047619047619048e-19

In [9]:
def est_viral_ratio(V, P, alpha, beta, S_ind, V_ind): 
    
    """
    Estimating the ratio of viruses to hosts given virus and host conven
    
    V     (float): viral abundance 
    P     (float): phytoplankton abundance
    alpha (float): conversion term for host related variables
    beta  (float): conversion term for virus related variables
    S_ind (float): Nucleotides per individual host
    V_ind (float): Nucleotides per individual virus
    """
    
    S_i = P / alpha / S_ind  #h_ind / per ml
    
    r = S_i * V_ind * beta / V #h_ind / v_ind 
    
    return r